# Optical Spectra Of Solids
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/periodic_systems/optical_spectra_of_solids/)

In this tutorial we will explore how to compute the optical properties of bulk systems from real-time TDDFT using **Octopus**. Two methods will be presented, one for computing the optical conductivity from the electronic current and another one computing directly the dielectric function using the so-called “gauge-field” kick method. Note that some of the calculations in this tutorial are quite heavy computationally, so they might require to run in parallel over a several cores to finish in a reasonable amount of time.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!mkdir 3-Optical-Spectra-Of-Solids

In [ ]:
cd 3-Optical-Spectra-Of-Solids

### Groundstate

#### Input
As always, we will start with the input file for the ground state, here of bulk silicon. In this case we will use a primitive cell of Si, composed of two atoms, as used in the [Getting started with periodic systems](1-getting-started.ipynb) tutorial:

In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs

PeriodicDimensions = 3

Spacing = 0.5

%LatticeVectors
 0.0 | 0.5 | 0.5
 0.5 | 0.0 | 0.5
 0.5 | 0.5 | 0.0
%

a = 10.18
%LatticeParameters
 a | a | a
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

nk = 2
%KPointsGrid
 nk  | nk  | nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes
%SymmetryBreakDir
 1 | 0 | 0
%


The description of most variables is given in the [Getting started with periodic systems](1-getting-started.ipynb). Here we only added one variable:

* [SymmetryBreakDir](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/symmetrybreakdir): this input variable is used here to specify that we want to remove from the list of possible symmetries to be used the ones that are broken by a perturbation along the x axis. Indeed, we will later apply a perturbation along this direction, which will break the symmetries of the system.

### Output 

Now run **Octopus** using the above input file. 

In [ ]:
!octopus

The important thing to note from the output is the effect of the [SymmetryBreakDir](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/symmetrybreakdir) input option:

In [ ]:
!cat stdout_gs.txt | grep -A 11 "[*] Symmetries [*]"

Here Octopus tells us that only 4 symmetries can be used, whereas without [SymmetryBreakDir](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/symmetrybreakdir), 24 symmetries could have been used. Since only some symmetries are used, twelve ‘‘k’'-points are generated, instead of two ‘‘k’'-points if we would have used the full list of symmetries. If we had not used symmetries at all, we would have 32 ‘‘k’'-points instead.

### Optical conductivity

We now turn our attention to the calculation of the optical conductivity. For this, we will apply to our system a time-dependent vector potential perturbation in the form of a Heaviside step function. This corresponds to applying a delta-kick electric field. However, because electric fields (length gauge) are not compatible with the periodicity of the system, we work here with vector potentials (velocity gauge).

The corresponding input file is given below:

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
FromScratch = yes
PeriodicDimensions = 3

Spacing = 0.5

%LatticeVectors
  0.0 | 0.5 | 0.5
  0.5 | 0.0 | 0.5
  0.5 | 0.5 | 0.0
%

a = 10.18
%LatticeParameters
 a | a | a
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

nk = 2
%KPointsGrid
  nk |  nk |  nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes
%SymmetryBreakDir
  1 | 0 | 0
%

ExperimentalFeatures = yes

%TDExternalFields
 vector_potential | 1.0 | 0.0 | 0.0 | 0.0 | "envelope_step"
%

%TDFunctions
 "envelope_step" | tdf_from_expr | "0.01*step(t)"
%

%TDOutput
 total_current
%

TDTimeStep = 0.3
TDPropagationTime = 1500
TDExponentialMethod = lanczos
TDExpOrder = 16

Most variables have already been discussed in the different prior tutorials. Here are the most important considerations:

* [TDOutput](https://www.octopus-code.org/documentation//13/variables/time-dependent/td_output/tdoutput): We ask for outputting the total electronic current. Without this, we will not be able to compute the optical conductivity.
* [TDExternalFields:](https://www.octopus-code.org/documentation//13/variables/time-dependent/tdexternalfields) We applied a vector potential perturbation along the ‘‘x’’ axis using.
* [TDFunctions](https://www.octopus-code.org/documentation//13/variables/time-dependent/tdfunctions): We defined a step function, an used a strength of the perturbation of 0.01. Note that the strength of the perturbation should be small enough, to guaranty that we remain in the linear response regime. Otherwise, nonlinear effects are coupling different frequencies and a an approach based of the “kick” method is not valid anymore.
Note that if you would have forgotten above to specify [SymmetryBreakDir](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/symmetrybreakdir), the code could would have stopped and let you know that your laser field is breaking some of the symmetries of the system:

******************************* FATAL ERROR ******************************* <br>
  \*\* Fatal Error (description follows) <br>
  \* In namespace Lasers: <br>
  \*--------------------------------------------------------------------  <br>
  \* The lasers break (at least) one of the symmetries used to reduce the k-points  .  <br>
  \* Set SymmetryBreakDir accordingly to your laser fields.  <br>
***********************************************************************

Now run **Octopus** using the above input file.

The time requiered to create all runs are in the order of 8 minutes.

In [ ]:
!octopus

After the calculation is finished, you can find in the td.general folder the total_current file, that contains the computed electronic current. From it, we want to compute the optical conductivity. This is done by running the utility oct-conductivity . Before doing so, we need to specify the type of broadening that we want to apply to our optical spectrum. In real time, this correspond to applying a damping function to the time signal. For instance, the Lorentzian broadening that we will employ in this tutorial is given by an exponential damping. To specify it, simply add the line


`PropagationSpectrumDampMode = exponential`

to your input file before running the oct-conductivity utility. Running the utility produces the file td.general/conductivity , which contains the optical conductivity 
σ(ω) of our system. However, note that at the moment the obtained conductivity is not scaled properly, it is normalized to the volume and proportional to the strength of the perturbation. Hence, in order to obtain the absorption spectrum, we need to compute

$ℑ[ϵ(ω)]= \frac{4\pi V}{E_0\omega} ℜ[σ(ω)]$ 

where ***V*** is the volume of the system (in this case 10.18^3/4) and E_0 
is the strength of the kick (here 0.01). Since our perturbation was 
along the ‘‘x’’ direction, we want to plot the ‘‘x’’ component of the conductivity. 
Therefore, in the end, we need to plot the second column of td.general/conductivity multiplied by 
appropriate factor versus the energy (first column). You can see how the spectrum should look like in Fig. 1. 
Because we are computing a current-current response, the spectrum exhibits a divergence at low frequencies. 
This numerical artifact vanishes when more ‘‘k’'-points are included in the simulation.

In [ ]:
%%writefile inp

CalculationMode = td
FromScratch = yes
PeriodicDimensions = 3

Spacing = 0.5

%LatticeVectors
  0.0 | 0.5 | 0.5
  0.5 | 0.0 | 0.5
  0.5 | 0.5 | 0.0
%

a = 10.18
%LatticeParameters
 a | a | a
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

nk = 2
%KPointsGrid
  nk |  nk |  nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes
%SymmetryBreakDir
  1 | 0 | 0
%

ExperimentalFeatures = yes

%TDExternalFields
 vector_potential | 1.0 | 0.0 | 0.0 | 0.0 | "envelope_step"
%

%TDFunctions
 "envelope_step" | tdf_from_expr | "0.01*step(t)"
%

%TDOutput
 total_current
%

TDTimeStep = 0.3
TDPropagationTime = 1500
TDExponentialMethod = lanczos
TDExpOrder = 16


PropagationSpectrumDampMode = exponential

In [ ]:
!oct-conductivity

In [ ]:
const = np.pi * 10.18**3 / 0.01

Looking at the optical spectrum, we observe three main peaks. These correspond to the $E_0$, $E_1$, and $E_1$ critical points of the bandstructure of silicon. Of course, the shape of the spectrum is not converged because we used too few ‘‘k’'-points to sample the Brillouin zone. Remember that you should also perform a convergence study of the spectrum with respect with the relevant parameters. In this case those would be the spacing and the ‘‘k’'-points.

In [ ]:
df = pd.read_csv(
    "td.general/conductivity",
    skiprows=4,
    usecols=[0, 1],
    names=["energy", "conductivity"],
    engine="python",
    sep="\s{4}|(?<=\s)-\s?",
)

df["absorption"] = const * df["conductivity"].values / (df["energy"].values + 10**-8)

df.plot(x="energy", y="absorption", xlim=[0, 1], ylim=[-20, 200]);

# Dielectric function using gauge field kick

Let us now look at a different way to compute the dielectric function of a solid. The dielectric function can be defined from the link between the macroscopic total electric field (or vector potential) and the external electric field. Thus, one can solve the Maxwell equation for computing the macroscopic induced field in order to get the total vector potential acting on the system.

This is the reasoning behind the gauge-field method, as proposed in Ref [$^1$](#first_reference).

The corresponding input file is given below:



In [ ]:
%%writefile inp 

stdout = 'stdout_die_gs.txt'
stderr = 'stderr_die_gs.txt'

CalculationMode = gs

PeriodicDimensions = 3

Spacing = 0.5

%LatticeVectors
 0.0 | 0.5 | 0.5
 0.5 | 0.0 | 0.5
 0.5 | 0.5 | 0.0
%

a = 10.18
%LatticeParameters
 a | a | a
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

nk = 5
%KPointsGrid
 nk  | nk  | nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes
%SymmetryBreakDir
 1 | 0 | 0
%

ExtraStates = 1
%Output
 dos
%

In [ ]:
!octopus

In [ ]:
%%writefile inp 

stdout = 'stdout_die_td.txt'
stderr = 'stderr_die_td.txt'

CalculationMode = td
FromScratch = yes

PeriodicDimensions = 3

Spacing = 0.5

%LatticeVectors
  0.0 | 0.5 | 0.5
  0.5 | 0.0 | 0.5
  0.5 | 0.5 | 0.0
%

a = 10.18
%LatticeParameters
 a | a | a
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

nk = 5
%KPointsGrid
  nk |  nk |  nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes
%SymmetryBreakDir
  1 | 0 | 0
%

%GaugeVectorField
 0.1 | 0 | 0
%

TDTimeStep = 0.2
TDPropagationTime = 1500
TDExponentialMethod = lanczos
TDExpOrder = 16

The time requiered to execute the following run are at least in the order of 120 minutes.

In [ ]:
!octopus

In [ ]:
!oct-dielectric-function

Note that before running this input file, you need to redo a GS calculation setting ``nk = 5``, to have the same ‘‘k’'-point grid as in the above input file.

Compared to the previous TD calculation, we have made few changes:

* [GaugeVectorField](https://www.octopus-code.org/documentation//13/variables/hamiltonian/gaugevectorfield): This activates the calculation of the gauge field and its output. This replaces the laser field used in the previous method.
* [KPointsGrid](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointsgrid): We used here a much denser ‘‘k’'-point grid. This is needed because of the numerical instability of the gauge-field method for very low number of ‘‘k’'-points.

After running this input file with **Octopus**, we can run the utility oct-dielectric-function , which produces the output td.general/dielectric_function . In this case we are interested in the imaginary part of the dielectric function along the ‘‘x’’ direction, which correspondents to the third column of the file. You can see how the spectra looks like in Fig. 2. As we can see, below the bandgap, we observe some numerical noise. This is again due to the fact that the spectrum is not properly converged with respect to the number of ‘‘k’'-points and vanishes if sufficient ‘‘k’'-points are used. Apart from this, we now obtained a much more converged spectrum, with the main three features starting to resemble the fully-converged spectrum.

In [ ]:
df = pd.read_csv(
    "td.general/dielectric_function",
    usecols=[0, 2],
    names=["energy", "absorption"],
    skiprows=1,
    sep="\s{3}|(?<=\s)s?",
    engine="python",
)


df["energy"] = df["energy"] * 27.2114


df.plot(
    x="energy",
    y="absorption",
    label=False,
    ylim=[-500, 500],
    title="Absorption spectrum of Si obtained using the gauge-field method.",
);

[Go to *4-Band-structure-unfolding.ipynb*](4-Band-structure-unfolding.ipynb)

1. Bertsch, G. F. and Iwata, J.-I. and Rubio, Angel and Yabana, K., Real-space, real-time method for the dielectric function, [Phys. Rev. B](https://doi.org/10.1103/PhysRevB.62.7998) 62 7998 (2000);
<span id="first_reference"></span>